In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
# load data
HOME = '../../'
data = pd.concat([pd.read_csv("{}new_data/input_train.csv".format(HOME), index_col=0), pd.read_csv('{}new_data/input_test.csv'.format(HOME), index_col=0)])
out_data = pd.concat([pd.read_csv('{}new_data/output_train-1.csv'.format(HOME), index_col=0, header=0), pd.read_csv('{}new_data/output_test-1.csv'.format(HOME), index_col=0, header=0)])
data = pd.concat([data, out_data], axis=1)

In [3]:
# load gene names
with open('{}gene labels/input_genes.txt'.format(HOME)) as f:
    genes = [gene.strip() for gene in f]
    
with open('{}gene labels/output_genes-1.txt'.format(HOME)) as f:
    out_genes = [gene.strip() for gene in f]

In [4]:
# calculate Pearson's correlation
corr = data[genes + out_genes].corr()

# Zero out self-correlations
np.fill_diagonal(corr.values, 0)

NameError: name 'gene' is not defined

In [6]:
# get 50 highest correlations for each gene
network = dict()
for gene in genes + out_genes:
    network[gene] = corr[gene].nlargest(50, keep='all')

In [7]:
# filter out genes with <= 0.0 correlation
network = {gene: list(interact.index[interact > 0.0]) for gene, interact in network.items()}

In [8]:
# keep only reciprocal correlations
filtered = dict()
for gene, interacts in network.items():
    filtered[gene] = list()
    for interact in interacts:
        if gene in network[interact]:
            filtered[gene].append(interact)

In [9]:
# convert adjacency list to edge list
edges = list()
for gene, interacts in filtered.items():
    edges += [(gene, interact) for interact in interacts if gene != interact]

In [10]:
with open('../networks/full_network.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(edges)